In [ ]:
import torch.nn as nn

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, init_output = 64):
        super(Discriminator,self).__init__()
        conv_params = {'kernel_size': 3, 'stride': 1, 'padding': 1}
        maxpool_params = {'kernel_size': 2, 'stride': 2, 'padding': 1}
        self.init_output = init_output
        
        self.layers = []
        
        in_channels = 1
        in_features = -1
        width = 186
        height = 171
        
        # this assumes you can mix conv and maxpools, with all fc at the end
        def conv(out_channels):
            self.layers.add(nn.Sequential(
                nn.Conv2d(in_channels, out_channels, **conv_params),
                nn.BatchNorm2d(num_features = out_channels),
                nn.LeakyReLU(0.2)
            ))
            in_channels = out_channels
            
        def maxpool():
            self.layers.add(nn.MaxPool2d(**maxpool_params))
            width = math.ceil(width /  2)
            height = math.ceil(height /  2)
            
            in_features = width * height * in_channels
            
        def fc(out_features):
            self.layers.add(nn.Sequential(
                nn.Linear(in_features, out_features),
                nn.Sigmoid()
            ))
            in_features = out_features
        
        conv(64)
        maxpool()
        conv(128)
        maxpool()
        conv(256)
        conv(256)
        maxpool()
        conv(512)
        conv(512)
        maxpool()
        conv(512)
        conv(512)
        maxpool()
        fc(4096)
        fc(4096)
        fc(100)
        
        
    def forward(self,X):
        for layer in self.layers:
            X = layer(X)
        X = X.view(-1,self.init_output*8*16)
        X = self.output(X)
        return X